# Name: Luke Pratley

# Testing Basic UNet

In [2]:
import os
import numpy as np

import tensorflow.keras 
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

from datetime import datetime


import PIL
import matplotlib.pyplot as plt
import pandas as pd

import math

import glob
import sys

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

sys.path.append('..')

import building_road_segmentation.unet_factory as unet_factory
import building_road_segmentation.optimization_factory as optimization_factory
import building_road_segmentation.data_generator as data_generator
import building_road_segmentation.loss_functions as loss_functions

Num GPUs Available:  1


In [3]:
image_dir =  glob.glob("..\\data_cleaning_EDA\\final_images_small\\*AOI_2*") + glob.glob("..\\data_cleaning_EDA\\final_images_small\\*AOI_3*") + glob.glob("..\\data_cleaning_EDA\\final_images_small\\*AOI_4*") + glob.glob("..\\data_cleaning_EDA\\final_images_small\\*AOI_5*")
mask_dir =  glob.glob("..\\data_cleaning_EDA\\final_masks_small\\*AOI_2*") + glob.glob("..\\data_cleaning_EDA\\final_masks_small\\*AOI_3*") + glob.glob("..\\data_cleaning_EDA\\final_masks_small\\*AOI_4*") + glob.glob("..\\data_cleaning_EDA\\final_masks_small\\*AOI_5*")
for p in range(len(image_dir)):
    s1 = image_dir[p].split('\\')[-1].replace('RGB-PanSharpen_', '').replace('.png', '')
    s2 = mask_dir[p].split('\\')[-1].replace('RGB-PanSharpen_', '').replace('.npy', '')
    assert s1 == s2
assert len(mask_dir) == len(image_dir)
print(len(image_dir))

17331


In [4]:
train_image_dir, test_image_dir, train_mask_dir, test_mask_dir = train_test_split(image_dir, mask_dir, test_size=0.3, random_state=42)

In [5]:
train_data = data_generator.READ_AND_AUGMENT_DATA(train_image_dir, train_mask_dir, batch_size=64)
test_data = data_generator.READ_DATA(test_image_dir, test_mask_dir, batch_size=128)

In [6]:
losses = {'dice_loss' : loss_functions.weighted_dice_loss([1, 1]), 'BinaryCrossentropy': loss_functions.weighted_binary_crossentropy(np.array([1, 1]))}

In [7]:
model_configurations = {'efficient_model_1': {'unet_levels': 4, 'number_of_start_kernels': 32, 'pooling_amount': 2, 'residual': False, 'loss': 'dice_loss'},
                        'efficient_model_2': {'unet_levels': 4, 'number_of_start_kernels': 32, 'pooling_amount': 2, 'residual': False, 'loss': 'BinaryCrossentropy'},
                        'model_1': {'unet_levels': 4, 'number_of_start_kernels': 32, 'pooling_amount': 2, 'residual': False, 'loss': 'dice_loss'},
                        'model_2': {'unet_levels': 4, 'number_of_start_kernels': 32, 'pooling_amount': 2, 'residual': False, 'loss': 'BinaryCrossentropy'},
                        'res_model_1': {'unet_levels': 4, 'number_of_start_kernels': 32, 'pooling_amount': 2, 'residual': True, 'loss': 'dice_loss'},
                        'res_model_2': {'unet_levels': 4, 'number_of_start_kernels': 32, 'pooling_amount': 2, 'residual': True, 'loss': 'BinaryCrossentropy'},
                        'attention_model_1': {'unet_levels': 4, 'number_of_start_kernels': 32, 'pooling_amount': 2, 'residual': False, 'loss': 'dice_loss'},
                        'attention_model_2': {'unet_levels': 4, 'number_of_start_kernels': 32, 'pooling_amount': 2, 'residual': False, 'loss': 'BinaryCrossentropy'},
                       }
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

In [8]:
len(model_configurations)

8

In [ ]:
for model_name, model_config in model_configurations.items():
    print(model_name, model_config)
    tf.keras.backend.clear_session()
    if not os.path.exists(model_name):
        if model_name.split('_')[0] =='efficient':
                    unet_model = unet_factory.EfficientNetUNet(efficientnet=tensorflow.keras.applications.efficientnet.EfficientNetB4,
                                            number_of_categories=2,
                                            unet_levels=model_config['unet_levels'],
                                            number_of_start_kernels=model_config['number_of_start_kernels'],
                                            kernel_shape=(3, 3),
                                            activation='relu',
                                            final_activation='sigmoid',
                                            pooling_amount=model_config['pooling_amount'],
                                            dropout_rate=0.2, residual=model_config['residual'])
        if model_name.split('_')[0] =='attention':
            unet_model = unet_factory.BasicUNet(number_of_categories=2,
                                            unet_levels=model_config['unet_levels'],
                                            number_of_start_kernels=model_config['number_of_start_kernels'],
                                            kernel_shape=(3, 3),
                                            activation='relu',
                                            final_activation='sigmoid',
                                            pooling_amount=model_config['pooling_amount'],
                                            dropout_rate=0.2, residual=model_config['residual'])
        else:
            unet_model = unet_factory.BasicUNet(number_of_categories=2,
                                            unet_levels=model_config['unet_levels'],
                                            number_of_start_kernels=model_config['number_of_start_kernels'],
                                            kernel_shape=(3, 3),
                                            activation='relu',
                                            final_activation='sigmoid',
                                            pooling_amount=model_config['pooling_amount'],
                                            dropout_rate=0.2, residual=model_config['residual'])

    else:
        unet_model = tf.keras.models.load_model(model_name, custom_objects={model_config['loss']: losses[model_config['loss']]})
        history_pre = pd.read_csv(model_name +'_history.csv', index_col=0)
    modelcheckpoint = tf.keras.callbacks.ModelCheckpoint(filepath=model_name, 
                        monitor='val_loss',
                        mode='min',
                        restore_best_weights=True, save_format='tf')
    unet_model.compile(optimizer=optimizer, loss=losses[model_config['loss']], metrics=[loss_functions.masked_accuracy()])
    history = unet_model.fit(train_data, epochs=50, validation_data=test_data, callbacks=[modelcheckpoint])
    if os.path.exists(model_name):
        history = pd.concat((history_pre , pd.DataFrame(history.history)), axis=0, ignore_index=True)
    history.to_csv(f'{model_name}_history.csv')  
    break

efficient_model_1 {'unet_levels': 4, 'number_of_start_kernels': 32, 'pooling_amount': 2, 'residual': False, 'loss': 'dice_loss'}
Epoch 1/50
948/948 [==============================] - 321s 335ms/step - loss: 0.5010 - accuracy: 0.8445 - val_loss: 0.4474 - val_accuracy: 0.8843
Epoch 2/50
948/948 [==============================] - 318s 335ms/step - loss: 0.4261 - accuracy: 0.8856 - val_loss: 0.4591 - val_accuracy: 0.8973
Epoch 3/50
948/948 [==============================] - 324s 341ms/step - loss: 0.4034 - accuracy: 0.8935 - val_loss: 0.4029 - val_accuracy: 0.8995
Epoch 4/50
948/948 [==============================] - 322s 340ms/step - loss: 0.3905 - accuracy: 0.8980 - val_loss: 0.4563 - val_accuracy: 0.9026
Epoch 5/50
948/948 [==============================] - 333s 351ms/step - loss: 0.3798 - accuracy: 0.9013 - val_loss: 0.3766 - val_accuracy: 0.9041
Epoch 6/50
948/948 [==============================] - 325s 343ms/step - loss: 0.3721 - accuracy: 0.9038 - val_loss: 0.3643 - val_accuracy: 0.

In [ ]:
histories = {}
count = 0
fig, ax = plt.subplots(1, 2, sharex=True, sharey=False, figsize=(10,5))
for model_name, model_config in model_configurations.items():
    if os.path.exists(model_name):
        history = pd.read_csv(model_name +'_history.csv', index_col=0)
        histories[model_name] = history
        if np.max(history['val_loss']) < 0.5:    
            ax[0].plot(history['val_accuracy'], label=model_name)
            ax[0].set_ylabel('val accuracy')
            ax[0].set_xlabel('epoch')
            ax[1].plot(history['val_loss'])
            ax[1].set_ylabel('val loss')
            ax[1].set_xlabel('epoch')
ax[0].legend()
plt.show()

In [ ]:
histories['efficient_model_1']

In [ ]:
X_test = np.array([np.array(PIL.Image.open(im)).astype(float) for im in test_image_dir])

In [ ]:
y_test = np.array([
            np.load(file_name)
               for file_name in test_mask_dir])

In [ ]:
model_metrics = {}
for k, (model_name, model_config) in enumerate(model_configurations.items()):
    if os.path.exists(model_name + '.h5'):
        model_metrics[model_name] = {'road_iou': {'AOI_2_Vegas': [], 'AOI_3_Paris': [], 'AOI_4_Shanghai' : [], 'AOI_5_Khartoum': []}, 
                                 'building_iou': {'AOI_2_Vegas': [], 'AOI_3_Paris': [], 'AOI_4_Shanghai' : [], 'AOI_5_Khartoum': []}}
        tf.keras.backend.clear_session()
        final_model = tf.keras.models.load_model(model_name + '.h5', custom_objects={model_config['loss']: losses[model_config['loss']]})
        print(model_name, model_config)
        y_pred = final_model.predict(X_test/255)
        y_pred = np.where(y_pred > 0.5, 1, 0)
        for k in range(y_test.shape[0]):
            for key in model_metrics[model_name]['road_iou'].keys():
                if key in test_mask_dir[k]:
                    if np.sum(y_test[k, :, :, 0]) > 0:
                        model_metrics[model_name]['road_iou'][key].append(loss_functions.intersection_over_union(y_test[k, :, :, 0], y_pred[k, :, :, 0]))
                    if np.sum(y_test[k, :, :, 1]) > 0:
                        model_metrics[model_name]['building_iou'][key].append(loss_functions.intersection_over_union(y_test[k, :, :, 1], y_pred[k, :, :, 1]))

In [ ]:
def plot_metrics(model_metrics, model_name):
    fig, ax = plt.subplots(1, 4, sharex=True, sharey=True, figsize=(10, 2))
    plt.suptitle(f'Road Average IoU per Image ({model_name})')
    ax = np.ravel(ax)
    for k, key in enumerate(model_metrics[model_name]['road_iou'].keys()):
        model_metrics[model_name]['road_iou'][key] = np.array(model_metrics[model_name]['road_iou'][key])
        ax[k].hist(model_metrics[model_name]['road_iou'][key], density=True, alpha=0.5, bins=np.arange(0, 1.05, 0.05), label=key, color=['green', 'red', 'navy', 'purple'][k])
        ax[k].legend()
        ax[k].set_xlabel('IoU')
        ax[k].axvline(np.mean(model_metrics[model_name]['road_iou'][key]))
    ax[0].set_ylabel('Density')
    plt.show()

    fig, ax = plt.subplots(1, 4, sharex=True, sharey=True, figsize=(10, 2))
    fig.suptitle(f'Building Average IoU per Image ({model_name})')
    ax = np.ravel(ax)
    for k, key in enumerate(model_metrics[model_name]['building_iou'].keys()):
        model_metrics[model_name]['building_iou'][key] = np.array(model_metrics[model_name]['building_iou'][key])
        ax[k].hist(model_metrics[model_name]['building_iou'][key], density=True, alpha=0.5, bins=np.arange(0, 1.05, 0.05), label=key, color=['green', 'red', 'navy', 'purple'][k])
        ax[k].legend()
        ax[k].set_xlabel('IoU')
        ax[k].axvline(np.mean(model_metrics[model_name]['building_iou'][key]))
    ax[0].set_ylabel('Density')
    plt.show()

In [ ]:
for model_name in model_configurations.keys():
    if os.path.exists(model_name):
        if np.mean(histories[model_name]['val_loss']) < 0.5:
            print(model_name, model_configurations[model_name])
            plot_metrics(model_metrics, model_name)

In [ ]:
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch

def plot_result(X_test, y_test, model_name, display_prob, index=0):
        final_model = tf.keras.models.load_model(model_name, custom_objects={model_config['loss']: losses[model_config['loss']]})
        y_pred = final_model.predict(X_test)
        print(np.ceil(y_test[k, :, :, 0][y_test[k, :, :, 0] !=0]).mean())
        if not display_prob:
            y_pred = np.where(y_pred > 0.5, 1, 0)
        fig, ax = plt.subplots(2, 3, figsize=(15, 8), sharex=True, sharey=True)
        ax[1, 0].set_title('Mismatch')
        ax[1, 0].imshow(np.sum(np.ceil(np.abs(y_test[index, :, :, :] - y_pred[index, :, :, :])),axis=-1), cmap=ListedColormap(['navy','red']))
        ax[1, 0].legend(handles=[Patch(facecolor='red', edgecolor='red', label='Mismatch'), 
                                 Patch(facecolor='navy', edgecolor='navy', label='Match')], loc="best")
        ax[0, 0].set_title('Labels')
        ax[0, 0].imshow(y_test[index, :, :, 0] * 10 + y_test[index, :, :, 1] * 20, cmap = ListedColormap(['navy','green','purple']))
        ax[0, 0].legend(handles=[Patch(facecolor='navy', edgecolor='navy',
                         label='Background'), Patch(facecolor='green', edgecolor='green',
                         label='Road Network'), Patch(facecolor='purple', edgecolor='purple',
                         label='Building')], loc="best")
        ax[1, 1].set_title('True Road Network')
        ax[1, 1].imshow(y_test[index, :, :, 0], vmin = 0, vmax=1)
        ax[1, 2].set_title('True Building Footprint')
        ax[1, 2].imshow(y_test[index, :, :, 1], vmin = 0, vmax=1)
        ax[0, 1].set_title('Predicted Road Network')
        ax[0, 1].imshow(y_pred[index, :, :, 0], vmin = 0, vmax=1)
        ax[0, 2].set_title('Predicted Building Footprint')
        ax[0, 2].imshow(y_pred[index, :, :, 1], vmin = 0, vmax=1)

In [ ]:
k = 40

batch = math.floor(k / test_data.batch_size) 
index = k - batch * test_data.batch_size
print(test_data.x[k])
print(test_data.y[k])
X_test_batch, y_test_batch = test_data.__getitem__(batch)
plt.figure(figsize=(5, 5))
plt.imshow(X_test_batch[index, :, :])
for model_name, model_config in model_configurations.items():
    print(model_config)
    if os.path.exists(model_name):
        plot_result(X_test_batch, y_test_batch, model_name=model_name, display_prob=True, index=index)
        plt.show()